In [2]:
import sys

In [10]:
sys.path.append('/Users/jessie_liu/anaconda/envs/py35/lib/python3.5/site-packages/')

In [7]:
!pip3 install nibabel

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!which python

/Users/jessie_liu/anaconda/envs/py35/bin/python


In [3]:
!python --version

Python 3.5.3 :: Anaconda 4.4.0 (x86_64)


In [11]:
import os
import sys
import numpy as np
import nibabel as nib
import matplotlib.pylab as plt
%matplotlib inline
import pandas as pd

In [2]:
CCA_pos = np.loadtxt('CCA1_pos_unmapped.txt')
CCA_neg = np.loadtxt('CCA1_neg_unmapped.txt')

In [12]:
# load group ICA map for 200 nodes
img = nib.load('../rawdata/groupICA/melodic_IC_sum.nii.gz')
data = img.get_fdata()

In [14]:
data.shape

(91, 109, 91, 200)

In [33]:
np.percentile(data[:,:,:,:],90)

0.60423267483711207

In [4]:
# Flip the node if the peak value is negative, and nomalise by the peak value
data_n = data.copy()
for i in range(200):
    if np.max(data[:,:,:,i]) < np.abs(np.min(data[:,:,:,i])):
        print(i)
        data_n[:,:,:,i] = -data_n[:,:,:,i]
    data_n[:,:,:,i] = np.divide(data_n[:,:,:,i], np.max(data[:,:,:,i]))

In [5]:
data_n[data_n<0] = 0 # why?

In [6]:
# map cca loadings to group ICA map
pos_map = data_n.copy()
neg_map = data_n.copy()
for i in range(200):
    pos_map[:,:,:,i] = data_n[:,:,:,i] * CCA_pos[i]
    neg_map[:,:,:,i] = data_n[:,:,:,i] * CCA_neg[i]

In [7]:
# sum 200 nodes to generate one positve and one negative map
mean_map = np.mean(data_n,axis=3)

Sum_pos_map = np.sum(pos_map,axis=3)
Sum_neg_map = np.sum(neg_map,axis=3)

In [8]:
pos = Sum_pos_map
neg = Sum_neg_map

pos[mean_map>0] = np.log(Sum_pos_map[mean_map>0]/mean_map[mean_map>0])
neg[mean_map>0] = np.log(Sum_neg_map[mean_map>0]/mean_map[mean_map>0])

In [9]:
# export
img_map = nib.Nifti1Image(pos, img.affine)
nib.save(img_map, 'CCA1_pos_20_unmapped.nii')
img_map = nib.Nifti1Image(neg, img.affine)
nib.save(img_map, 'CCA1_neg_20_unampped.nii')